## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/jennifermadson/Desktop/Data-Analytics-BootCamp/School-Projects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Lodwar,KE,3.1191,35.5973,84.25,42,15,10.40,few clouds
1,1,Port Alfred,ZA,-33.5906,26.8910,60.40,63,2,18.81,clear sky
2,2,Dikson,RU,73.5069,80.5464,8.06,87,99,11.90,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,33.46,93,75,18.41,snow
4,4,San Vicente De Canete,PE,-13.0751,-76.3835,65.28,77,19,8.05,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you'd like for your trip? "))
max_temp = float(input("What is the maximum temperature you'd like for your trip? "))


What is the minimum temperature you'd like for your trip? 75
What is the maximum temperature you'd like for your trip? 90


In [25]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

vacation_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]


In [18]:
# 4a. Determine if there are any empty rows.
vacation_cities_df.isnull().sum()

City_ID        0
City           0
Country        2
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [31]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = vacation_cities_df.dropna()


In [32]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Lodwar,KE,84.25,few clouds,3.1191,35.5973,
5,Natal,BR,81.25,few clouds,-5.7950,-35.2094,
13,Tukrah,LY,76.21,scattered clouds,32.5341,20.5791,
16,Salalah,OM,75.29,clear sky,17.0151,54.0924,
20,Buchanan,LR,77.23,moderate rain,5.8808,-10.0467,


In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():   
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"  
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    # 6e. Make request and retrieve the JSON data from the search.  
    hotels = requests.get(base_url, params=params).json()
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

except (IndexError):
    print("Hotel not found... skipping.")

In [43]:
#7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df



,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
684,Mabaruma,GY,75.49,moderate rain,8.2,-59.7833,Monty Brooms Guest House


In [44]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(10.0, 21.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))